In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time


In [2]:
def get_g_stats(soup):
    teams = soup.find('h1').text.split(' @ ')
    home = teams[1]
    away = teams[0]
    date = soup.find('h2').text.split('\n')[0]

    g = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}stgall')})

    gs = []


    df = pd.read_html(str(g[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'
    gs.append(df)
    df = pd.read_html(str(g[5]))[0]
    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'
    gs.append(df)
    gs_master = pd.concat(gs)
    gs_master['Date'] = date
    gs_master = gs_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'TOI', 'Goals Against', 'Expected Goals Against']]
    return gs_master
    

In [3]:
def get_rel_stats(soup):
    teams = soup.find('h1').text.split(' @ ')
    home = teams[1]
    away = teams[0]
    date = soup.find('h2').text.split('\n')[0]

    ind = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}stall')})
    oi = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}oi5v5')})
    ois = []
    inds = []

    df = pd.read_html(str(oi[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'

    ois.append(df)
    df = pd.read_html(str(oi[1]))[0]
    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'

    ois.append(df)

    df = pd.read_html(str(ind[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'

    inds.append(df)
    df = pd.read_html(str(ind[1]))[0]

    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'

    inds.append(df)

    oi_master = pd.concat(ois)
    oi_master['Date'] = date
    oi_master = oi_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'TOI', 'GF', 'GA', 'xGF', 'xGA']]
    ind_master = pd.concat(inds)
    ind_master['Date'] = date
    ind_master = ind_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'Position', 'TOI', 'Goals', 'First Assists', 'Second Assists', 'ixG', 'Shots Blocked', 'Total Penalties', 'Penalties Drawn', 'Faceoffs Won', 'Faceoffs Lost']]

    df = ind_master.merge(oi_master, on=['Player', 'Team'])
    return df.drop(columns = ['Date_y', 'Venue_y', 'TOI_y', 'Opposition_y'])

In [8]:
from datetime import datetime
for year in [2022,]:
    print(year, "started at ", datetime.now())
    goalie_dfs = []
    dfs = []
    gameid = 20001
    url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    dfs.append(get_rel_stats(soup))
    goalie_dfs.append(get_g_stats(soup))
    time.sleep(30)
    timer = 0
    errors = []
    while len(soup.text) > 3000:
        try:
            gameid += 1
            if timer == 100:
                time.sleep(661)
                timer = 0
            url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
            req = requests.get(url)
            soup = BeautifulSoup(req.content)
            stats = get_rel_stats(soup)
            dfs.append(get_rel_stats(soup))
            goalie_dfs.append(get_g_stats(soup))
            time.sleep(30)
            timer += 1
        except:
            errors.append(gameid)
    pd.concat(dfs).to_csv(f'{year}{year+1}_skaters.csv', index=False)
    pd.concat(goalie_dfs).to_csv(f'{year}{year+1}_goalies.csv', index=False)
    print(year, errors)

2022 started at  2023-03-12 13:06:17.182190
2022 [20320, 20547]


In [4]:
from datetime import datetime
for year in [2022,]:
    dfs = []
    goalie_dfs = []
    print(year, "started at ", datetime.now())
    gameid = 21135
    url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    dfs.append(get_rel_stats(soup))
    goalie_dfs.append(get_g_stats(soup))
    time.sleep(30)
    timer = 0
    errors = []
    while len(soup.text) > 3000:
        try:
            gameid += 1
            if timer == 100:
                time.sleep(661)
                timer = 0
            url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
            req = requests.get(url)
            soup = BeautifulSoup(req.content)
            stats = get_rel_stats(soup)
            dfs.append(get_rel_stats(soup))
            goalie_dfs.append(get_g_stats(soup))
            time.sleep(30)
            timer += 1
        except:
            errors.append(gameid)

    print(year, errors)

2022 started at  2023-03-23 16:04:55.859094
2022 [21135, 21136, 21137, 21138, 21139, 21140, 21141, 21142, 21143, 21144, 21145, 21146, 21147]


In [22]:
catted = pd.concat(dfs)
catted.columns = ['Date', 'Player', 'Venue', 'Team', 'Opposition', 'Position',
       'TOI', 'Goals', 'First Assists', 'Second Assists', 'ixG',
       'Shots Blocked', 'Total Penalties', 'Penalties Drawn', 'Faceoffs Won',
       'Faceoffs Lost', 'GF', 'GA', 'xGF', 'xGA']

In [29]:
skaters = pd.read_csv(f'cleaned/{year}{year+1}_skaters.csv')
skaters['Season'] = str(year) + str(year+1)
skaters['Player'] = skaters['Player'].str.replace('\xa0', ' ')
skaters['Player'] = skaters['Player'].str.replace('Tim Stützle', 'Tim Stutzle')
skaters['Player'] = skaters['Player'].str.replace('Alexis Lafrenière', 'Alexis Lafreniere')

skaters['Date'] = skaters['Date'].str.replace('/', '-')
skaters = skaters.loc[:, ~skaters.columns.str.contains('^Unnamed')]

catted['Season'] = str(year) + str(year+1)
catted['Player'] = catted['Player'].str.replace('\xa0', ' ')
catted['Player'] = catted['Player'].str.replace('Tim Stützle', 'Tim Stutzle')
catted['Player'] = catted['Player'].str.replace('Alexis Lafrenière', 'Alexis Lafreniere')

catted['Date'] = catted['Date'].str.replace('/', '-')
catted = catted.loc[:, ~catted.columns.str.contains('^Unnamed')]

skater_master = pd.concat([skaters, catted])
skater_master.to_csv('cleaned/20222023_skaters.csv')

In [3]:
import plotly.graph_objects as go
import numpy as np
from moepy import lowess
from collections import namedtuple
import pickle
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor
from scipy.stats import norm
from datetime import datetime
import pandas as pd
import numpy as np
from numpy import random
from scipy.stats import norm
skaters = pd.read_csv(f'cleaned/master_skaters.csv')
skaters['exGF'] = skaters['TOI']*0.032+0.055
skaters['exGA'] = skaters['TOI']*0.032+0.055
skaters['eGF'] = skaters['TOI']*0.032+0.055
skaters['eGA'] = skaters['TOI']*0.032+0.055
skaters['GFAA'] = skaters['GF'] - skaters['eGF']
skaters['GAAA'] =  skaters['eGA'] - skaters['GA']
skaters['xGFAA'] = skaters['xGF'] - skaters['exGF']
skaters['xGAAA'] =  skaters['exGA'] - skaters['xGA']
skaters['Production'] = (0.75*skaters['Goals']) + (0.7*skaters['First Assists']) + (0.55*skaters['Second Assists']) + (0.5*skaters['ixG']) + (0.05*skaters['Shots Blocked']) 
skaters['Misc'] = (0.15*(skaters['Penalties Drawn'] - skaters['Total Penalties'])) + (0.01*(skaters['Faceoffs Won'] - skaters['Faceoffs Lost']))
skaters['Offense'] = np.where(skaters['Position'] == 'D', 
((0.425*(skaters['GF'] + skaters['GFAA'])) + (1.7*(skaters['xGF'] + skaters['xGFAA'])))/2.125 - 0.75,
(((0.625*(skaters['GF'] + skaters['GFAA'])) + (0.625*(skaters['xGF'] + skaters['xGFAA'])))/2) - 0.4)
skaters['Defense'] = np.where(skaters['Position'] == 'D', 
(((0.575*(-skaters['GA'] + skaters['GAAA'])) + (2.3*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 1.05, 
(((0.4375*(-skaters['GA'] + skaters['GAAA'])) + (1.7*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 0.6)
skaters['Score'] = skaters['Production'] + skaters['Misc'] + skaters['Offense'] + skaters['Defense']
skaters = skaters.loc[:, ~skaters.columns.str.contains('^Unnamed')]
births = pd.read_csv('birthdates.csv')
merged = skaters.merge(births[['Player', 'Season', 'dob']], how='left', on=['Player', 'Season'])
merged.drop_duplicates(subset=['Player', 'Team', 'Date', 'Opposition', 'Season'], inplace=True)
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team == 'New York Islanders')),'1999-07-26', merged['dob'])
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team != 'New York Islanders')),'1996-02-17', merged['dob'])
merged[["Date", "dob"]] = merged[["Date", "dob"]].apply(pd.to_datetime)
merged['Age'] = (merged["Date"] - merged["dob"]).dt.days/365
merged['RoundAge'] = np.floor(merged["Age"])


start_dates = {season: datetime.strptime((str(season)[:4] + '-10-04'), '%Y-%m-%d') for season in skaters.groupby(['Season']).count().index}
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: f'𝒩(μ={s[0]:.3f}, 𝜎²={s[1]:.3f})'
def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain

    x = x + K*y      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)

def get_projections(player, time_type):
    alpha = 0.99809326
    df = merged[merged.Player == player]
    df = df.sort_values(by='Date')
    df['Career Game'] = range(1, len(df) + 1)
    df['Adjusted'] = df['Score']*(50/1.55) + 50
    df[["Date", "dob"]] = df[["Date", "dob"]].apply(pd.to_datetime)
    projections = [0]
    scores = np.concatenate((np.zeros(10),  df['Score'].to_list()))
    total = 0

    for i in range(-9, len(df)):
        curr = 0

        j = i-1
        if j >= 0:

            weight = alpha**(df['Date'].to_list()[i] - df['Date'].to_list()[j]).days
        else:
            weight = alpha**(-1*j)
        
        curr += weight * projections[-1] * total
        curr += weight * scores[j+10]
        total += 1
        total *= weight
        projections.append((curr/total))            

        

    df['Projection'] = projections[10:]



    sensor_var = 1.3330171065772254
    process_var = 0.0013513089896074557
    process_model = gaussian(0., process_var)
    pos = gaussian(0., 100.)
    N = len(df) + 10
    zs = np.concatenate((np.zeros(10), df['Score'].to_list()))
    ps = []
    for i in range(N):
        prior = predict(pos, process_model)    
        pos = update(prior, gaussian(zs[i], sensor_var))
        ps.append(pos.mean)
    df['Kalman'] = ps[10:]

    # load the dataset
    series = df[['Projection', 'Kalman']]
    values = series.values

    # fit model
    model = pickle.load(open('finalized_model.sav', 'rb'))
    # construct an input for a new preduction
    preds = df['Kalman'].to_list()[:82]
    for i in range(82, len(df)):
        row = values[i-82:i].flatten()
    # make a one-step prediction
        yhat = model.predict(asarray([row]))
        preds.append(yhat[0])
    df['XGB'] = preds

    ages = range(18, 48)
    A = [-0.14396,-0.05957,0.021176,0.080782,0.144998,0.176731,
            0.181012,0.179595,0.162485,0.137085,0.114395,0.094328,
            0.071203,0.043198,0.013411,0.002706,-0.01087,-0.030415,
            -0.04996,-0.03732,-0.06303,-0.03521,-0.038033904,-0.033721952,
            -0.02941,-0.04036,-0.05131,-0.08806,-0.08892,-0.08978]
    poly  = np.polyfit(ages, A, 4)
    for i in range(1, len(preds)):
        prev_age = df['Age'].to_list()[i-1]
        prev_expected = np.polyval(poly, prev_age)
        curr_expected = np.polyval(poly, df['Age'].to_list()[i])
        multiplier = curr_expected/prev_expected
        if preds[i] < 0:
            preds[i] = preds[i]/multiplier
        else:
            preds[i] = preds[i] * multiplier



    for i in range(0, len(df)):
        season = df['Season'].to_list()[i]
        since_start = (df['Date'].to_list()[i] - start_dates[season]).days
        season_adjust = (-0.00109 * since_start  + 1.1)
        if df['Venue'].to_list()[i] == 'Home':
            preds[i] = preds[i]*1.35*season_adjust*1.2
        else:
            preds[i] = preds[i]*1.35*season_adjust*0.8


        prev = merged[(merged.Season == season) & (merged.Date < df['Date'].to_list()[i])]
        opps = prev.groupby(['Opposition']).mean(numeric_only=True)[['Score']]
        opps['Multiplier'] = (opps['Score']+5)/5.438829866884756
        mult = opps[opps.index == (df['Opposition'].to_list()[i])]['Multiplier']
        if len(mult) > 0:
            mult = mult[0]
        else:
            mult = 1
        preds[i] = preds[i] * mult

    df['adjXGB'] = preds
      
    # Model fitting
    frac = 0.25
    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, num_fits=20, robust_iters=3)
    df['XGB_Lowess'] = df_quantiles[0.5].to_list()

    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, num_fits=20, robust_iters=3)
    df['adjXGB_Lowess'] = df_quantiles[0.5].to_list()
    pots = []
    mults = []
    for i in range(0, len(df)):
        curr_age = df['Age'].to_list()[i]
        if curr_age <= 24.26795064:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = (0.37233423)/(curr_expected+0.2)
            mults.append(multiplier)
            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)

        elif curr_age <= 38.62152643:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
        elif curr_age <= 42.25458411:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = 0.24220954/(curr_expected+0.2)
            mults.append(multiplier)

            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)        
        else:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
    df['multiplier'] = mults
    df['Potential'] = pots
    df.reset_index(inplace=True, drop=True)
    df_quantiles.reset_index(inplace=True, drop=True)
    return df, df_quantiles



In [4]:
players = list(merged[merged.Season == 20222023].Player.unique())
players.sort()
recents = []
errors = []
for player in players[300:600]:
    print(player)
    try:
        df = pd.read_csv('projections/20222023/'+(player.replace(' ', '_'))+'.csv')
    except:
        df, df_quantiles = get_projections(player, 'Career Game')
        df.to_csv('projections/20222023/'+(player.replace(' ', '_'))+'.csv')
        recents.append(df[df.index == len(df)-1])

Gustav Lindstrom


100%|██████████| 9/9 [00:01<00:00,  5.22it/s]


Gustav Nyquist


100%|██████████| 9/9 [00:04<00:00,  2.06it/s]


Gustav Olofsson


...

100%|██████████| 9/9 [00:03<00:00,  2.48it/s]


Mikhail Sergachev


100%|██████████| 9/9 [00:04<00:00,  2.23it/s]


Mikko Rantanen


100%|██████████| 9/9 [00:03<00:00,  2.30it/s]
